<a href="https://colab.research.google.com/github/takeru0208/sample_rnn/blob/master/sample_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hello world")

hello world


In [5]:
%cd /content/drive/My Drive/Colab Notebooks
!git config --global user.email "aikonntatto@gmail.com"
!git config --global user.name "Takeru Tsuchiya"

/content/drive/My Drive/Colab Notebooks
